In [1]:
## The following notebook outlines how the data was grabbed and cleaned to prepare it for analysis.

In [2]:
# The following program cleans and then outputs the cleaned broadband data.
# The data was originally grabbed as an excel file from the following link, then saved as a csv.
#    https://techdatasociety.asu.edu/broadband-data-portal/dataaccess/countydata

# Dependencies
import pandas as pd

# Paths to be used
import_path = "CSV_files/broadband_long2000-2018rev.csv"
output_path = "CSV_files/clean_broadband.csv"

In [3]:
# rows containing any invalid values are dropped.
broadband_df = pd.read_csv(import_path)
clean_df = broadband_df.dropna(how = "any")
clean_df = clean_df.rename(columns={"statenam" : "State",
                                    "county":"County",
                                    "year": "Year",
                                   "id":"GEO ID",
                                   "broadband":"Broadband (%)",
                                   "cfips":"CFIPS"})
clean_df

,State,County,Year,GEO ID,Broadband (%),CFIPS
17,Alabama,Autauga County,2017,0500000US01001,0.618182,1001
18,Alabama,Autauga County,2018,0500000US01001,0.789000,1001
26,Alabama,Baldwin County,2007,0500000US01003,0.588521,1003
27,Alabama,Baldwin County,2008,0500000US01003,0.591518,1003
28,Alabama,Baldwin County,2009,0500000US01003,0.594515,1003
...,...,...,...,...,...,...
59488,Wyoming,Uinta County,2018,0500000US56041,0.882000,56041
59506,Wyoming,Washakie County,2017,0500000US56043,0.622951,56043
59507,Wyoming,Washakie County,2018,0500000US56043,0.783000,56043
59525,Wyoming,Weston County,2017,0500000US56045,0.586207,56045


In [4]:
# Check that all broadband access data is in an acceptable range
print(f'The maximum value is {clean_df["Broadband (%)"].max()}')
print(f'The minimum value is {clean_df["Broadband (%)"].min()}')

The maximum value is 0.97388148
The minimum value is 0.01125842


In [5]:
# Drop all duplicate values, which will be one's with the same year and location
clean_df.drop_duplicates(["Year", "GEO ID"])

,State,County,Year,GEO ID,Broadband (%),CFIPS
17,Alabama,Autauga County,2017,0500000US01001,0.618182,1001
18,Alabama,Autauga County,2018,0500000US01001,0.789000,1001
26,Alabama,Baldwin County,2007,0500000US01003,0.588521,1003
27,Alabama,Baldwin County,2008,0500000US01003,0.591518,1003
28,Alabama,Baldwin County,2009,0500000US01003,0.594515,1003
...,...,...,...,...,...,...
59488,Wyoming,Uinta County,2018,0500000US56041,0.882000,56041
59506,Wyoming,Washakie County,2017,0500000US56043,0.622951,56043
59507,Wyoming,Washakie County,2018,0500000US56043,0.783000,56043
59525,Wyoming,Weston County,2017,0500000US56045,0.586207,56045


In [6]:
# Clean dataframe is output as a csv file.
clean_df.to_csv(output_path, index=False)

In [7]:
## Now that the broadband data is in a usable format, the census data needs to be grabbed.  The following does so for
##    exactly one NAICS label.

In [8]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import requests
from pprint import pprint

# Import csv broadband data to dataframe
broadband_df = pd.read_csv("CSV_files/clean_broadband.csv")

# Census API Key
from config import census_api_key

In [9]:
# Assume year is 2017
# The census database is the following:
#    https://www.census.gov/data/developers/data-sets/economic-census.html
# The documentation used is the following:
#    https://api.census.gov/data/2017/ecnbasic/variables.html

# Craft base url
base_url = "https://api.census.gov/data/2017/ecnbasic"

# Craft query url
query_url = base_url + f"?get=NAICS2017_LABEL,EMP,NAME,GEO_ID&for=county:*&NAICS2017=51&key={census_api_key}"

# Grabs relevant data from census
response = requests.get(query_url).json()

In [10]:
# Initialize empty arrays
NAICS2017_label = []
number_employed = []
county_name = []
GEO_ID = []

# Creates arrays by appending relevant data to array,which maintains index.
for county in response:
    NAICS2017_label.append(county[0])
    number_employed.append(county[1])
    county_name.append(county[2])
    GEO_ID.append(county[3])
    
# Arrays are compiled into a dataframe
county_2017_df = pd.DataFrame({"NAICS2017 Label":NAICS2017_label,
                              "Number Employed":number_employed,
                              "County, State":county_name,
                              "GEO ID":GEO_ID})

# As the first row of the dataframe is the labels from the Census, it must be dropped.
county_2017_df.drop(index=0,inplace=True)

# Output the resulting array.
county_2017_df.head()

,NAICS2017 Label,Number Employed,"County, State",GEO ID
1,Information,0,"Benton County, Tennessee",0500000US47005
2,Information,199,"Warren County, Tennessee",0500000US47177
3,Information,1424,"Washington County, Tennessee",0500000US47179
4,Information,131,"Weakley County, Tennessee",0500000US47183
5,Information,0,"Campbell County, Tennessee",0500000US47013


In [11]:
# 2017 Broadband dataframe is crafted only grabbing relevant columns to prevent clutter
broadband_2017_df = broadband_df.loc[broadband_df["Year"] == 2017][["Broadband (%)","GEO ID","Year"]]

# Broadband dataframe is combined with county 2017 data from census.  Inner merge is used to drop all empty data points.
combined_2017_df = county_2017_df.merge(broadband_2017_df, how="inner", on="GEO ID")

# Data is output to screen and as a csv.
combined_2017_df.to_csv(f'CSV_files/{combined_2017_df["Year"].min()}_{combined_2017_df["NAICS2017 Label"].min()}_ecnbasic.csv',
                        index=False)
combined_2017_df.head()

,NAICS2017 Label,Number Employed,"County, State",GEO ID,Broadband (%),Year
0,Information,0,"Benton County, Tennessee",0500000US47005,0.561538,2017
1,Information,199,"Warren County, Tennessee",0500000US47177,0.567398,2017
2,Information,1424,"Washington County, Tennessee",0500000US47179,0.858978,2017
3,Information,131,"Weakley County, Tennessee",0500000US47183,0.628906,2017
4,Information,0,"Campbell County, Tennessee",0500000US47013,0.552326,2017


In [12]:
## Of course, this method isn't particularly efficient at grabbing lots of data, so it was turned into a for loop to run
##    through lots of NAICS labels

In [13]:
# List of single labels to examine for all that the census has data on that doesn't crash.
# This was done by looking at census documentation, writing out each NAICS label into a list, and removing values that crashed
# The original list was [21,22,23,31,32,33,42,44,45,48,49,51,52,53,54,55,56,61,62,71,72,81]
NAICS_list = [22,51,52,53,54,56,61,62,71,72,81]

# Assume year is 2017
# Grab csv for acs5 in 2017 then cut out all irrelevant columns
acs5_data = pd.read_csv("CSV_files/acs5_2017.csv")
acs5_df = acs5_data[["Name","Population"]]


# Craft base url
base_url = "https://api.census.gov/data/2017/ecnbasic"

# Empty list of file paths and total number employed in label are made
NAICS_2017_csv_paths = []

# for loop runs through each label generating a csv of the relevant dataframe.
for label in NAICS_list:
    
    # As this label is printed first, if a csv is crafted, the next number appears. Otherwise, the last label crashed it.
    print(label)
 
    # query url is crafted
    query_url = base_url + f"?get=NAICS2017_LABEL,EMP,NAME,GEO_ID&for=county:*&NAICS2017={label}&key={census_api_key}"
   
    # census is called
    response = requests.get(query_url).json()
    
    # Initialize empty arrays
    NAICS2017_label = []
    number_employed = []
    county_name = []
    GEO_ID = []

    # Creates arrays by appending relevant data to array,which maintains index.
    for county in response:
        NAICS2017_label.append(county[0])
        number_employed.append(county[1])
        county_name.append(county[2])
        GEO_ID.append(county[3])
    
    # Arrays are compiled into a dataframe
    county_2017_data = pd.DataFrame({"NAICS2017 Label":NAICS2017_label,
                                  "Number Employed":number_employed,
                                  "Name":county_name,
                                  "GEO ID":GEO_ID})
    
    
    # Populaton data is merged into dataframe.
    county_2017_df = pd.DataFrame.merge(county_2017_data ,acs5_df, how="inner")

    # As the first row of the dataframe is the labels from the Census, it must be dropped.
    county_2017_df.drop(index=0,inplace=True)
    
    # 2017 Broadband dataframe is crafted only grabbing relevant columns to prevent clutter
    broadband_2017_df = broadband_df.loc[broadband_df["Year"] == 2017][["Broadband (%)","GEO ID","Year"]]

    # Broadband dataframe is combined with county 2017 data from census.  Inner merge is used to drop all empty data points.
    combined_2017_df = county_2017_df.merge(broadband_2017_df, how="inner", on="GEO ID")


    # Column needed for math is saved as int to ensure math and not concatenation
    # Total number employed in label is appended to relevant array
    # New column for percent employed in label is calculated then made into new column for combined dataframe
    combined_2017_df["Number Employed"] = combined_2017_df["Number Employed"].astype(float)
    combined_2017_df["Percent Employed"] = combined_2017_df["Number Employed"] / combined_2017_df["Population"]
    
    # Data is saved as a csv.
    combined_2017_df.to_csv(f'CSV_files/{combined_2017_df["Year"].min()}_{combined_2017_df["NAICS2017 Label"].min()}_ecnbasic.csv',
                            index=False)
    
    # File path for csv file just written to is appended to relevant array
    NAICS_2017_csv_paths.append(f'CSV_files/{combined_2017_df["Year"].min()}_{combined_2017_df["NAICS2017 Label"].min()}_ecnbasic.csv')

    
    # Did the loop finish?
    print("Successfully retrieved")
    
# List of csv paths is turned into a dataframe then saved as a csv.
NAICS_2017_csv_paths_df = pd.DataFrame({"Paths": NAICS_2017_csv_paths})
NAICS_2017_csv_paths_df.to_csv(f'CSV_files/NAICS_2017_csv_paths.csv', index=False) 

22
Successfully retrieved
51
Successfully retrieved
52
Successfully retrieved
53
Successfully retrieved
54
Successfully retrieved
56
Successfully retrieved
61
Successfully retrieved
62
Successfully retrieved
71
Successfully retrieved
72
Successfully retrieved
81
Successfully retrieved
